In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from PIL import Image

In [4]:
data_path = './slovo/data'

DATA_PATH = './sequence_data'

# Чтение аннотационных файлов
annotations_df = pd.read_csv(os.path.join('slovo','annotations.csv'), sep='\t')

actions_annotations_df = pd.read_csv(os.path.join('actions_annotations.csv'), sep=';')

actions = np.unique(np.array(actions_annotations_df['action']))

# actions = np.append(actions, 'none')

gestures = np.array(actions_annotations_df['text'])

# количество кадров в видео
sequence_length = 30

print(actions, len(actions))

['down' 'right' 'shake' 'static' 'to_fist' 'up' 'z_write'] 7


In [5]:
# Подсчёт среднего количества кадров в видео из датасета
gesture_count = 0
total_length = 0

for gesture in gestures:

    # срез данных для текущего жеста
    gestures_row = annotations_df[annotations_df['text'] == gesture]
    for index, row in gestures_row.iterrows():
        length = int(row['length'])

        total_length += length
        gesture_count += 1

print(f'Всего видео в датасете: {gesture_count}\nСреднее количество кадров: {total_length/gesture_count}')

Всего видео в датасете: 1700
Среднее количество кадров: 45.32411764705882


In [6]:
# создание папки для датасета
try:
    os.makedirs(os.path.join(DATA_PATH))
except:
    pass

for action in actions:
    sequence = 0

    # создание папки для action
    try:
        os.makedirs(os.path.join(DATA_PATH, action))
    except:
        pass

    # срез данных для текущего действия
    actions_rows = actions_annotations_df[actions_annotations_df['action'] == action]
    for index, row in actions_rows.iterrows():
        gesture = row['text']

        # срез данных для текущего жеста
        gestures_row = annotations_df[annotations_df['text'] == gesture]
        for index, row in gestures_row.iterrows():
            attachment_id = row['attachment_id']
            length = int(row['length'])

            # Путь к видеофайлу
            video_path = os.path.join(data_path, attachment_id + '.mp4')

            # Проверка существования видеофайла
            if not os.path.exists(video_path):
                print("Видеофайл {} не найден".format(video_path))
                continue

            # создание папки для последовательности кадров
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            except:
                pass

            # Чтение видеофайла
            cap = cv2.VideoCapture(video_path)

            image = None
            frame_count = 0

            for frame_num in range(length):
                # Read frame
                ret, frame = cap.read()

                if ret == False:
                    continue
                
                # 10 кадров в начале обрезаются и обрезаются 5 последних кадров
                if frame_num >= 40 or frame_num < 10:
                    continue

                frame_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_count) + '.jpg')
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(rgb_frame)
                image.save(frame_path)

                frame_count += 1

            # дублирование последнего кадра при малом количестве кадров
            length_tmp = length
            while length_tmp < 40:
                frame_path = os.path.join(DATA_PATH, action, str(sequence), str(length_tmp - 10) + '.jpg')
                image.save(frame_path)
                length_tmp += 1

            sequence += 1
            cap.release()


In [7]:
# запись своего датасета
DATA_PATH = './sequence_data'

no_sequences = 60 # количество видео в датасете у каждого класса
sequence_length = 30 # количество кадров в последовательности

cap = cv2.VideoCapture(0)

action = 'down'

# Loop through sequences aka videos
for sequence in range(no_sequences):
    # Loop through video length aka sequence length
    for frame_num in range(sequence_length):

        if frame_num == 0:
            cv2.waitKey(2000)

        # Read feed
        ret, frame = cap.read()
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
        cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        # Show to screen
        cv2.imshow('OpenCV Feed', frame)

        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

        frame_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num) + '.jpg')
        image = Image.fromarray(rgb_frame)
        image.save(frame_path)                

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
                    
cap.release()
cv2.destroyAllWindows()

In [23]:
cap.release()
cv2.destroyAllWindows()